In [ ]:
import time
from datetime import date
import pandas as pd
import os
import glob
import pycytominer
import sys
import CBE_utils as CBE
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.spatial.distance import correlation
import re
import gc

In [ ]:
import importlib
importlib.reload(CBE)

In [ ]:
input_path = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/output/"
output_path = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/results/"

annotation_dir = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis_revision/annotation/"

In [ ]:
# get folder list 
# load for each site the normalized files
# reduce the features selecting the correct feature list
folders = [name for name in os.listdir(input_path) if os.path.isdir(os.path.join(input_path, name))]

In [ ]:
pattern = "[A-Z][0-9][0-9][0-9][0-9]_R[1-4]_mad_robustize_reduced-corr.csv"

FMP_Data = []
IMTM_Data = []
MEDINA_Data = []
USC_Data = []

for folder in folders:

    site_specific_path = os.path.join(input_path, folder)

    # load normalized data
    file_list = glob.glob(site_specific_path + os.sep + '*' + os.sep + pattern, recursive=True)

    for file in file_list:

        filename = os.path.basename(file)
    
        try:
        
            Data_Temp = pd.read_csv(file)
            row_count = Data_Temp.shape[0]
        
            print(f"File: {filename} has {row_count} rows")
        
            if folder == 'FMP':
                
                FMP_Data.append(Data_Temp)

            elif folder == 'IMTM':

                IMTM_Data.append(Data_Temp)

            elif folder == 'MEDINA':

                MEDINA_Data.append(Data_Temp)

            elif folder == 'USC':

                USC_Data.append(Data_Temp)
            
        except Exception as e:
        
            print(f"Error reading file {filename}: {e}")
        
        


In [ ]:
### concat all files together
FMP_Data_aggregated = pd.concat(FMP_Data)
FMP_Data_aggregated = FMP_Data_aggregated.reset_index(drop = True)
print("Aggregated Data has shape ", FMP_Data_aggregated.shape)

In [ ]:
IMTM_Data_aggregated = pd.concat(IMTM_Data)
IMTM_Data_aggregated = IMTM_Data_aggregated.reset_index(drop = True)
print("Aggregated Data has shape ", IMTM_Data_aggregated.shape)

In [ ]:
MEDINA_Data_aggregated = pd.concat(MEDINA_Data)
MEDINA_Data_aggregated = MEDINA_Data_aggregated.reset_index(drop = True)
print("Aggregated Data has shape ", MEDINA_Data_aggregated.shape)

In [ ]:
USC_Data_aggregated = pd.concat(USC_Data)
USC_Data_aggregated = USC_Data_aggregated.reset_index(drop = True)
print("Aggregated Data has shape ", USC_Data_aggregated.shape)

In [ ]:
Data_aggregated = IMTM_Data_aggregated

In [ ]:
Data_aggregated.head()

In [ ]:
# get unique Metadata_RoughID
def merge_if_eos_cpd(row):

    if row['Metadata_RoughID'] == 'EOS_cpd':

        return f"{row['Metadata_plate_name']}_{row['Metadata_Well']}"
    
    else:

        return row['Metadata_RoughID']
    

Data_aggregated['Metadata_RoughID_unique'] = Data_aggregated.apply(merge_if_eos_cpd, axis=1)

In [ ]:
Data_aggregated.head()

# Paper Version

In [ ]:
len(Data_aggregated)

In [ ]:
# Filter for plates with four replicates

replicate_list = Data_aggregated['Metadata_plate_map_name'].unique()
replicate_dataframe = pd.DataFrame(replicate_list, columns=['Metadata_plate_map_name']) 

replicate_list_newcolumns = replicate_dataframe['Metadata_plate_map_name'].str.split('_', n=1, expand=True)
replicate_dataframe['Metadata_plate_name'] = replicate_list_newcolumns[0]
replicate_dataframe['Metadata_replicate_number'] = replicate_list_newcolumns[1]

replicate_counts = replicate_dataframe.groupby('Metadata_plate_name')['Metadata_replicate_number'].count().reset_index()

replicate_counts 

In [ ]:
# Group by 'measurement_code' and filter groups that have exactly the specified number of replicates
filtered_replicate_dataframe= replicate_dataframe.groupby('Metadata_plate_name').filter(lambda x: len(x['Metadata_replicate_number']) == 4)
filtered_replicate_dataframe = filtered_replicate_dataframe.sort_values(by=['Metadata_plate_name'])

Data_aggregated_filtered = Data_aggregated[Data_aggregated['Metadata_plate_map_name'].isin(filtered_replicate_dataframe['Metadata_plate_map_name'])]

In [ ]:
Data_aggregated_filtered.head()
print(Data_aggregated_filtered['Metadata_source'].unique())
print(Data_aggregated_filtered['Metadata_plate_name'].unique())

## Toxicity filter

In [ ]:
HepG2_Reduced_Tox, HepG2_Reduced_Tox_Cond = CBE.remove_tox(
    Data_aggregated_filtered, 
    key_col = ["Metadata_RoughID_unique", "Metadata_plate_name"], 
    SD_Threshold = 2.5,  
    plot_distribution = True)

In [ ]:
len(HepG2_Reduced_Tox["Metadata_RoughID_unique"].unique())

## Raw %Replication

In [ ]:
HepG2_replicating, HepG2_corr_replicating_df = CBE.remove_non_reproducible(
    Data_aggregated_filtered, 
    n_samples = 10000, 
    n_replicates = 4, 
    ID_col = "Metadata_RoughID_unique", 
    description = "Data_50")

In [ ]:
n_experiments = len(HepG2_corr_replicating_df)

plt.rcParams['figure.facecolor'] = 'white' # Enabling this makes the figure axes and labels visible in PyCharm Dracula theme
plt.figure(figsize=[12, n_experiments*6])

for i in range(n_experiments):
    plt.subplot(n_experiments, 1, i+1)
    plt.hist(HepG2_corr_replicating_df.loc[i,'Null_Replicating'], label='non-replicates', density=True, bins=20, alpha=0.5)
    plt.hist(HepG2_corr_replicating_df.loc[i,'Replicating'], label='replicates', density=True, bins=20, alpha=0.5)
    plt.axvline(HepG2_corr_replicating_df.loc[i,'Value_95'], label='95% threshold')
    plt.legend(fontsize=20)
    plt.title(
        f"{HepG2_corr_replicating_df.loc[i,'Description']}\n" +
        f"Percent Replicating = {HepG2_corr_replicating_df.loc[i,'Percent_Replicating']}",
        fontsize=25
    )
    plt.ylabel("density", fontsize=25)
    plt.xlabel("Batch correlation", fontsize=25)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    sns.despine()
plt.tight_layout()

plt.savefig(output_path + str(date.today()) + "_Percent_Replicating.pdf", 
            transparent=False, 
            bbox_inches='tight', 
            dpi = 600)

In [ ]:
len(HepG2_replicating["Metadata_RoughID_unique"].unique())

## Activity filter

In [ ]:
Data_aggregated_filtered["Metadata_plate_name"]

In [ ]:
Data_aggregated_filtered["Metadata_RoughID_unique"]

In [ ]:
## gets feature vector
Features_HepG2_Norm_Reduced = CBE.get_feature_vector(Data_aggregated_filtered)

HepG2_Norm_Reduced_Median = pycytominer.consensus(
        profiles = Data_aggregated_filtered, # A file or pandas DataFrame of profile data
        replicate_columns = ["Metadata_RoughID_unique", "Metadata_plate_name", "Metadata_source"], # Metadata columns indicating which replicates to collapse, defaults to [“Metadata_Plate”, “Metadata_Well”]
        operation = "median", # (str) – The method used to form consensus profiles, defaults to “median”
        features = Features_HepG2_Norm_Reduced, # (str, list) – The features to collapse, defaults to “infer”
)

In [ ]:
len(HepG2_Norm_Reduced_Median["Metadata_RoughID_unique"].unique())

In [ ]:
key_col = ["Metadata_RoughID_unique", "Metadata_plate_name", "Metadata_source"]

HepG2_active, HepG2_low_active = CBE.remove_low_active(HepG2_Norm_Reduced_Median, 
                                                   key_col,
                                                   3.0, 
                                                   5.0)

print("HepG2 active:", len(HepG2_active["Metadata_RoughID_unique"].unique()))
print("HepG2 low-active:", len(HepG2_low_active["Metadata_RoughID_unique"].unique()))

In [ ]:
HepG2_Reduced_Tox_active = Data_aggregated_filtered[
    (Data_aggregated_filtered['Metadata_RoughID_unique'].isin(HepG2_active['Metadata_RoughID_unique']))]

In [ ]:
HepG2_Reduced_Tox_active["Metadata_RoughID_unique"].unique()

# Active %Replication

In [ ]:
HepG2_active_replicating, HepG2_active_corr_replicating_df = CBE.remove_non_reproducible(
    HepG2_Reduced_Tox_active, 
    n_samples = 10000, 
    n_replicates = 4, 
    ID_col = "Metadata_RoughID_unique", 
    description = "Data_50")

In [ ]:
# plot % replicating
corr_replicating_df = HepG2_active_corr_replicating_df

n_experiments = len(corr_replicating_df)


plt.rcParams['figure.facecolor'] = 'white' # Enabling this makes the figure axes and labels visible in PyCharm Dracula theme
plt.figure(figsize=[12, n_experiments*6])

for i in range(n_experiments):
    plt.subplot(n_experiments, 1, i+1)
    plt.hist(corr_replicating_df.loc[i,'Null_Replicating'], label='non-replicates', density=True, bins=20, alpha=0.5)
    plt.hist(corr_replicating_df.loc[i,'Replicating'], label='replicates', density=True, bins=20, alpha=0.5)
    plt.axvline(corr_replicating_df.loc[i,'Value_95'], label='95% threshold')
    plt.legend(fontsize=20)
    plt.title(
        f"{corr_replicating_df.loc[i,'Description']}\n" +
        f"Percent Replicating = {corr_replicating_df.loc[i,'Percent_Replicating']}",
        fontsize=25
    )
    plt.ylabel("density", fontsize=25)
    plt.xlabel("Batch correlation", fontsize=25)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    sns.despine()
plt.tight_layout()

plt.savefig(output_path + str(date.today()) + "_Percent_Replicating_Induction_Filter.pdf", 
            transparent=False, 
            bbox_inches='tight', 
            dpi = 600)# 

# Compute activity from single replicates

In [ ]:
Data_aggregated_filtered_R1 = Data_aggregated_filtered[Data_aggregated_filtered["Metadata_replicate_number"] == "R1"]

Data_aggregated_filtered_R1 


features = CBE.get_feature_vector(Data_aggregated_filtered_R1)
metadata_dataframe = set(Data_aggregated_filtered_R1.columns) - set(features)

In [ ]:
HepG2_active, HepG2_low_active = CBE.remove_low_active(Data_aggregated_filtered_R1, 
                                                   metadata_dataframe,
                                                   3.0, 
                                                   5.0)

print("HepG2 active:", len(HepG2_active["Metadata_RoughID_unique"].unique()))
print("HepG2 low-active:", len(HepG2_low_active["Metadata_RoughID_unique"].unique()))

In [ ]:
Data_aggregated_filtered_R2 = Data_aggregated_filtered[Data_aggregated_filtered["Metadata_replicate_number"] == "R2"]

HepG2_active, HepG2_low_active = CBE.remove_low_active(Data_aggregated_filtered_R2, 
                                                   metadata_dataframe,
                                                   3.0, 
                                                   5.0)

print("HepG2 active:", len(HepG2_active["Metadata_RoughID_unique"].unique()))
print("HepG2 low-active:", len(HepG2_low_active["Metadata_RoughID_unique"].unique()))

In [ ]:
Data_aggregated_filtered_R3 = Data_aggregated_filtered[Data_aggregated_filtered["Metadata_replicate_number"] == "R3"]

HepG2_active, HepG2_low_active = CBE.remove_low_active(Data_aggregated_filtered_R3, 
                                                   metadata_dataframe,
                                                   3.0, 
                                                   5.0)

print("HepG2 active:", len(HepG2_active["Metadata_RoughID_unique"].unique()))
print("HepG2 low-active:", len(HepG2_low_active["Metadata_RoughID_unique"].unique()))

In [ ]:
Data_aggregated_filtered_R4 = Data_aggregated_filtered[Data_aggregated_filtered["Metadata_replicate_number"] == "R4"]

HepG2_active, HepG2_low_active = CBE.remove_low_active(Data_aggregated_filtered_R4, 
                                                   metadata_dataframe,
                                                   3.0, 
                                                   5.0)

print("HepG2 active:", len(HepG2_active["Metadata_RoughID_unique"].unique()))
print("HepG2 low-active:", len(HepG2_low_active["Metadata_RoughID_unique"].unique()))

# Reproduce with randomly assinged identity

In [ ]:
Data_aggregated_filtered_copy = Data_aggregated_filtered.copy()

In [ ]:
# get unique Metadata_RoughID
def randomized_ID(row):

    if row['Metadata_RoughID'] == 'EOS_cpd':

        return f"{row['Metadata_plate_name']}_{row['Metadata_Well_randomized']}"
    
    else:

        return row['Metadata_RoughID']

In [ ]:
Data_aggregated_filtered_copy['Metadata_RoughID_randomized'] = Data_aggregated_filtered_copy.apply(randomized_ID, axis=1)

In [ ]:
Data_aggregated_filtered_copy.head()

In [ ]:
random_replicating, corr_random_replicating = CBE.remove_non_reproducible(
    Data_aggregated_filtered_copy, 
    n_samples = 10000, 
    n_replicates = 4, 
    ID_col = "Metadata_RoughID_randomized", 
    description = "Data_50")

In [ ]:
n_experiments = len(corr_random_replicating)

plt.rcParams['figure.facecolor'] = 'white' # Enabling this makes the figure axes and labels visible in PyCharm Dracula theme
plt.figure(figsize=[12, n_experiments*6])

for i in range(n_experiments):
    plt.subplot(n_experiments, 1, i+1)
    plt.hist(corr_random_replicating.loc[i,'Null_Replicating'], label='non-replicates', density=True, bins=20, alpha=0.5)
    plt.hist(corr_random_replicating.loc[i,'Replicating'], label='replicates', density=True, bins=20, alpha=0.5)
    plt.axvline(corr_random_replicating.loc[i,'Value_95'], label='95% threshold')
    plt.legend(fontsize=20)
    plt.title(
        f"{corr_random_replicating.loc[i,'Description']}\n" +
        f"Percent Replicating = {corr_random_replicating.loc[i,'Percent_Replicating']}",
        fontsize=25
    )
    plt.ylabel("density", fontsize=25)
    plt.xlabel("Batch correlation", fontsize=25)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    sns.despine()
plt.tight_layout()

plt.savefig(output_path + str(date.today()) + "_Percent_Replicating.pdf", 
            transparent=False, 
            bbox_inches='tight', 
            dpi = 600)

In [ ]:
## gets feature vector
Features = CBE.get_feature_vector(Data_aggregated_filtered_copy)

random_median = pycytominer.consensus(
        profiles = Data_aggregated_filtered_copy, # A file or pandas DataFrame of profile data
        replicate_columns = ["Metadata_RoughID_randomized"], # Metadata columns indicating which replicates to collapse, defaults to [“Metadata_Plate”, “Metadata_Well”]
        operation = "median", # (str) – The method used to form consensus profiles, defaults to “median”
        features = Features, # (str, list) – The features to collapse, defaults to “infer”
)

In [ ]:
key_col = ["Metadata_RoughID_randomized"]

active_random, low_active_random = CBE.remove_low_active(random_median, 
                                                   key_col,
                                                   2.0, 
                                                   5.0)

print("HepG2 active:", len(active_random["Metadata_RoughID_randomized"].unique()))
print("HepG2 low-active:", len(low_active_random["Metadata_RoughID_randomized"].unique()))

In [ ]:
active_random

In [ ]:
replicates_active_random = Data_aggregated_filtered_copy[
    (Data_aggregated_filtered_copy['Metadata_RoughID_randomized'].isin(active_random['Metadata_RoughID_randomized']))]

In [ ]:
replicates_active_random.head()

In [ ]:
len(replicates_active_random["Metadata_RoughID_randomized"])

In [ ]:
len(replicates_active_random["Metadata_RoughID_randomized"].unique())

In [ ]:
HepG2_active_replicating_random, HepG2_active_corr_replicating_df_random = CBE.remove_non_reproducible(
    replicates_active_random, 
    n_samples = 10000, 
    n_replicates = 4, 
    ID_col = "Metadata_RoughID_randomized", 
    description = "Data_50")

In [ ]:
# plot % replicating
corr_replicating_df = HepG2_active_corr_replicating_df_random

n_experiments = len(corr_replicating_df)


plt.rcParams['figure.facecolor'] = 'white' # Enabling this makes the figure axes and labels visible in PyCharm Dracula theme
plt.figure(figsize=[12, n_experiments*6])

for i in range(n_experiments):
    plt.subplot(n_experiments, 1, i+1)
    plt.hist(corr_replicating_df.loc[i,'Null_Replicating'], label='non-replicates', density=True, bins=20, alpha=0.5)
    plt.hist(corr_replicating_df.loc[i,'Replicating'], label='replicates', density=True, bins=20, alpha=0.5)
    plt.axvline(corr_replicating_df.loc[i,'Value_95'], label='95% threshold')
    plt.legend(fontsize=20)
    plt.title(
        f"{corr_replicating_df.loc[i,'Description']}\n" +
        f"Percent Replicating = {corr_replicating_df.loc[i,'Percent_Replicating']}",
        fontsize=25
    )
    plt.ylabel("density", fontsize=25)
    plt.xlabel("Batch correlation", fontsize=25)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    sns.despine()
plt.tight_layout()

plt.savefig(output_path + str(date.today()) + "_Percent_Replicating_Induction_Filter.pdf", 
            transparent=False, 
            bbox_inches='tight', 
            dpi = 600)# 